In [3]:
from dotenv import load_dotenv
import os, sys

load_dotenv('/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/.env')

# Add the parent directory to sys.path so you can import your modules from a subdirectory
sys.path.append(os.path.abspath('..'))

import rag
from rag import CONFIG

In [4]:
# Config LangSmith if you also want the traces
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain_evaluator.ipynb on ASK main/local"

In [5]:
from langsmith.evaluation import evaluate
from langsmith import Client

client = Client()

In [7]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_openai import ChatOpenAI


def prepare_cot_qa_data(run, example):
    '''
    Create a dictionary for the evaluator to use.

    run is the rag function 
    example is the example from the dataset
    '''
    return {
        "input": example.inputs["question"],
        "reference": example.outputs["ground_truth_answer"],
        "prediction": run.outputs["answer"],
    }


# Initialize the LLM with the desired model and temperature
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

accuracy_evaluator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": llm},
    prepare_data=prepare_cot_qa_data,
)

In [ ]:
# cot_qa uses the CotQAEvalChain class which uses the prompt template here: https://smith.langchain.com/hub/wfh/cot_qa
accuracy_evaluator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data=prepare_cot_qa_data,
)

In [8]:

dataset_name = "ASK-groundtruth-v2"
# ASK-groundtruth_v1   initial_EDA

split_name = "1_question"

data = dataset_name

# I don't think I need this one anymore
data = client.list_examples(dataset_name=dataset_name, splits=["1_question"])


experiment_prefix = "ASK_ART_eval-llm-gpt-4o-mini"

experiment_description = "Testing cost using gpt-4o-mini for Eval and gpt-4-turbo for RAG. This will run an eval over a signle question 1x. AppName-TestType-TestVariables. ART stands for Accuraacy, Recall, Truthfulness. oai= OpenAI model. accuracy, recall, truthfulness are the test variables."

In [9]:
def target_function(input: dict):
    '''maps the shape input from our example, which is a single-field dictionary, to the rag function we are testing, which accepts a string'''
    return rag.rag(input["question"])


evaluate(
    target_function,
    data=data,
    evaluators=[accuracy_evaluator],
    experiment_prefix=experiment_prefix,
    num_repetitions=1,
    metadata=CONFIG,
)  # type: ignore    # This supresses an error

View the evaluation results for experiment: 'ASK_ART_eval-llm-gpt-4o-mini-bfdb37e4' at:
https://smith.langchain.com/o/3941ecea-6957-508c-9f4f-08ed62dc7d61/datasets/0b24ff94-f4f0-4197-89f3-765f835936c9/compare?selectedSessions=2d064312-26e0-4c97-82bb-c74e03487782




0it [00:00, ?it/s]

2025-01-12 19:07:53.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:53.806 
  command:

    streamlit run /Users/drew_wilkins/Drews_Files/Drew/Python/Repositories/ASK/.venv-main/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-12 19:07:53.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:53.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:54.308 Thread 'Thread-8': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:54.311 Thread 'Thread-8': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:56.446 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-12 19:07:56.447 Thread 'MainThread': missing Scrip

,inputs.question,outputs.user_question,outputs.enriched_question,outputs.context,outputs.answer,outputs.llm_sources,error,reference.ground_truth_answer,reference.ground_truth_sources,feedback.COT Contextual Accuracy,execution_time,example_id,id
0,"How is harassment generally defined, and who i...","How is harassment generally defined, and who i...","How is harassment generally defined, and who i...",[page_content='COMDTINST M16790.1G \n \n \n \n...,Harassment is generally defined as unwelcome a...,[COMDTINST M16790.1G Section B. Anti-Discrimin...,None,Harassment is generally defined as unwelcome a...,[],1,9.132581,a3326a11-5c24-4ceb-a04e-5bb708dd9b38,651f0668-93f0-4159-b21f-9b5549e883d1
